###  Ybigta 19기 신입기수프로젝트 5조 와인추천시스템 중 Tokenizing부분입니다

In [1]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 1.8 MB/s 
     |████████████████████████████████| 448 kB 47.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import pandas as pd
import numpy as np
import konlpy
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt,Kkma
okt = Okt()
kkma = Kkma()
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
wine = pd.read_csv("/content/drive/My Drive/Colab Notebooks/total_.csv")  

In [6]:
wine.head()

,Unnamed: 0,name_ko,name_en,price,type,country,city,aroma,food,rate_expert,rate_consumer,alcohol,sugar,acidity,body,tannin,maker's_note,tasting_note,posting_title,posting_text,wine_title,wine_text,hashtag,blog_text
0,0,더 롱 리틀 독 스파클링,The Long Little Dog Sparkling,30000.0,스파클링,프랑스,랑그독 루씨용,"복숭아, 살구, 꽃, 흰꽃, 레몬","리코타, 파스타, 샐러드",-,3.5,NaN,1.0,5.0,2.0,1.0,"밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",NaN,NaN,NaN,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,1,루베롱 블랑,LUBERON BLANC,25000.0,NaN,프랑스,론,"헤이즐넛, 꽃, 아카시아, 레몬",NaN,-,4,NaN,1.0,3.0,3.0,1.0,"화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",NaN,NaN,NaN,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","Marie Stuart, Cuvee de la Reine Brut",145000.0,스파클링,프랑스,샹파뉴,꽃,"아페리티프, 파스타",-,4,12 %,1.0,4.0,3.0,1.0,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",NaN,와인21닷컴이 선정한 7월 이달의 와인,조금 늦게 찾아온 장마와 무더위가 기다리고 있는 7월. 장마철의 습한 기운을 떨쳐내...,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,3,"마스카 델 타코, 로시 피노 네로 로사토","Masca del Tacco, “ Ro’Si ” Pinot Nero Rosato",38000.0,NaN,이탈리아,뿔리아,"딸기, 미네랄",NaN,-,4,12~13 %,1.0,3.0,2.0,1.0,밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,NaN,NaN,NaN,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,4,"비에티, 바르베라 다스티 '트레 비그네'","Vietti, Barbera d'Asti 'Tre Vigne'",50000.0,NaN,이탈리아,아스티,"체리, 오크",NaN,-,4,NaN,1.0,3.0,3.0,4.0,바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,NaN,NaN,NaN,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [7]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     806 non-null    int64  
 1   name_ko        805 non-null    object 
 2   name_en        805 non-null    object 
 3   price          304 non-null    float64
 4   type           68 non-null     object 
 5   country        325 non-null    object 
 6   city           309 non-null    object 
 7   aroma          321 non-null    object 
 8   food           214 non-null    object 
 9   rate_expert    325 non-null    object 
 10  rate_consumer  325 non-null    object 
 11  alcohol        241 non-null    object 
 12  sugar          325 non-null    float64
 13  acidity        325 non-null    float64
 14  body           325 non-null    float64
 15  tannin         325 non-null    float64
 16  maker's_note   317 non-null    object 
 17  tasting_note   28 non-null     object 
 18  posting_ti

###### 먼저, 수치형 데이터 보다는 블로그나 기사 등을 통해 특징을 추출하는 것이 목표이기 때문에 해당 열들을 지웠습니다.

###### posting_text의 경우, 불필요한 말이 너무 많아 오히려 분석의 성능을 저하시킨다고 판단해 지웠습니다

In [8]:
del wine["Unnamed: 0"]
del wine["name_en"]
del wine["price"]
del wine["country"]
del wine["city"]
del wine["rate_expert"]
del wine["rate_consumer"]
del wine["sugar"]
del wine["alcohol"]
del wine["acidity"]
del wine["tannin"]
del wine["posting_text"]
del wine["food"]
del wine["body"]

In [9]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",NaN,NaN,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,NaN,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",NaN,NaN,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",NaN,와인21닷컴이 선정한 7월 이달의 와인,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",NaN,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,NaN,NaN,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",NaN,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,NaN,NaN,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


###### 분석의 성능을 위해 반복되는 불필요한 어구들을 정규표현식을 사용해 지웠습니다. (기자이름, 기사를 쓴 날, 평점 등)

In [10]:
import re

In [11]:
for i  in range(0, 806):
    wine.tasting_note[i] = re.sub("\n.{1,}\n[0-9]{4}[.][0-9]{2}[.][0-9]{2}\n[A-Z0-9]{2,}\s빈티지\n[0-9][.][0-9]"," ", str(wine.tasting_note[i]))

In [12]:
wine.posting_title.unique()

array([nan, '와인21닷컴이 선정한 7월 이달의 와인', '와인21추천 BEST OF BEST, 선물하기 좋은 와인',
       '올해 밸런타인데이에 함께할 와인!', '졸업 시즌의 축하주, 스파클링 와인',
       '와인21추천 BEST OF BEST, 꼭 한번 마셔볼만한 마트 와인', '초여름밤, 설레는 기분을 선사해줄 와인',
       "와인21추천 BEST OF BEST, 집에서 더 즐겁게 '홈파티 와인'", '늦여름에 마시기 좋은 레드 와인',
       '와인21추천 BEST OF BEST - 치명적 우아함, 부르고뉴(Bourgogne)',
       '[와인추천] 소소한 야외 파티에서 즐기기 좋은 와인', '와인21닷컴이 선정한 3월 이달의 와인',
       '부부의 날을 기념하며 마시기 좋은 와인', '와인21닷컴이 선정한 2월 이달의 와인',
       '호캉스에 단 한 병의 와인을 가지고 간다면?', '와인21닷컴이 선정한 5월 이달의 와인',
       '와인21닷컴이 선정한 6월 이달의 와인', '와인21닷컴이 선정한 8월 이달의 와인',
       '와인21닷컴이 선정한 9월 이달의 와인', '와인21닷컴이 선정한 10월 이달의 와인',
       '와인21닷컴이 선정한 11월 이달의 와인', '와인21닷컴이 선정한 12월 이달의 와인',
       '와인21닷컴이 선정한 2021년 1월 이달의 와인', '와인21닷컴이 선정한 4월 이달의 와인',
       '폭염이 지속되는 여름밤에 마시고 싶은 와인', '무더운 날 칠링해서 마시기 좋은 레드 와인은?',
       '삼계탕과 잘 어울리는 와인은?', '봄바람과 함께, 야외에서 즐기는 와인',
       '행사가 많은 5월, 선물이 고민인가요?', '춘곤증을 날려주는 청량감 넘치는 와인',
       '미세먼지가 심한 날, 홈술 추천 와인', '벚꽃 필 무렵엔 이 와인을!', '봄날, 지금 썸타는 중이라면?',
       "와인21's PI

In [13]:
for i  in range(0, 806):
    wine.posting_title[i] = re.sub("와인21추천"," ", str(wine.posting_title[i]))
    wine.posting_title[i] = re.sub("이달의 와인"," ", str(wine.posting_title[i]))
    wine.posting_title[i] = re.sub("BEST OF BEST"," ", str(wine.posting_title[i]))    
    wine.posting_title[i] = re.sub("와인"," ", str(wine.posting_title[i]))    
    wine.posting_title[i] = re.sub("21닷컴이 선정한"," ", str(wine.posting_title[i]))        

###### 불필요하다고 판단된 어구들이 지워진 이후 unique 값들을 확인해보았습니다

In [14]:
wine.tasting_note.unique()

array(['nan', ' \n세인트 넘 조아요~~~~\n\n굿뜨베리...', '  \n샤도네이를 좋아하게 해준 와인',
       ' \n양고기 허브솔트와 마리아주',
       ' \n오픈직후 피어오르는 잘익은 과실향이 인상적이나 한모금하는순간 할퀴는듯한 탄닌이 길들여지지 않은야생의 맹수같은 느낌이다. 몇년의 기간을 두고 마시면 깜짝놀랄만한 녀석으로 변할것같다고 조심스레 예상해본다.\n마시자마자 든 느낌은 아주 기름기가 많은 육류와 함께하면 좋을듯하다',
       ' \n진지하면서도 위트가 넘치는 흥미로운 와인. 오레곤을 제외한 미국와인의 특성상 75%이상 사용된 주품종은 레이블에 명시할 수 있는 와인법에 따라 분명 피노누아 이외의 다른 품종이 블렌딩 된 것 같은데 감을 잡을 수 없었다. 찾아보니 글로벌~한 블렌딩. 딸기, 섬세한 체다치즈, 바닐라, 캬라멜 등의 밝은 이미지의 아로마 뒤에 보여지는 우아한 빨레는 이러한 블렌딩의 마법인것 같기도 하다. 우연히 견과류와 함께했는데 크렌베리,캐슈넛과의 시너지 효과가 꽤나 좋았다. 호두는 껍질이 탄닌과 충돌해 에러:)',
       ' \n만들어 먹는 뱅쇼는 그때그때 맛도 다르고 잘못해서 끓기라도 하면 맛이 틀어지는데 크리스트 킨들스는 병째 데워 마시면 온기가 퍼지고 맛 역시 튀는곳 없이 무난해서 좋았다.\n마무리로 올라오는 상큼한 과일맛이 일품 \n전반적으로 산뜻하고 깔끔한 맛~\n너무 강하지도 약하지도 않은 바디감과 산미....매우 괜찮았음\n그냥 데워만 마셔도 좋겠지만 레몬 조각을 살짝 띄워서 마시니 더욱 맛있었는듯..^^',
       ' \n1804 tasting at coex expo \n1804 tasting at coex expo',
       ' \n차갑게 칠링한 로제 스파클링 와인은 특유의 청량감으로 몸과 마음에 신선한 기운을 불어넣어 준다. No.1 호주 와인 브랜드인 제이콥스 크릭이 만든 로제 스파클링은 제철 과일인 딸기나 참외 등과 잘 어울린다. 또, 캠핑에서 즐길 수 있는 조

In [15]:
wine.posting_title.unique()

array(['nan', '   7월  ', '   , 선물하기 좋은  ', '올해 밸런타인데이에 함께할  !',
       '졸업 시즌의 축하주, 스파클링  ', '   , 꼭 한번 마셔볼만한 마트  ',
       '초여름밤, 설레는 기분을 선사해줄  ', "   , 집에서 더 즐겁게 '홈파티  '",
       '늦여름에 마시기 좋은 레드  ', '    - 치명적 우아함, 부르고뉴(Bourgogne)',
       '[ 추천] 소소한 야외 파티에서 즐기기 좋은  ', '   3월  ', '부부의 날을 기념하며 마시기 좋은  ',
       '   2월  ', '호캉스에 단 한 병의  을 가지고 간다면?', '   5월  ', '   6월  ',
       '   8월  ', '   9월  ', '   10월  ', '   11월  ', '   12월  ',
       '   2021년 1월  ', '   4월  ', '폭염이 지속되는 여름밤에 마시고 싶은  ',
       '무더운 날 칠링해서 마시기 좋은 레드  은?', '삼계탕과 잘 어울리는  은?',
       '봄바람과 함께, 야외에서 즐기는  ', '행사가 많은 5월, 선물이 고민인가요?',
       '춘곤증을 날려주는 청량감 넘치는  ', '미세먼지가 심한 날, 홈술 추천  ', '벚꽃 필 무렵엔 이  을!',
       '봄날, 지금 썸타는 중이라면?', " 21's PICK, 카베르네 소비뇽(Cabernet Sauvignon)편",
       '봄을 기다리며 즐기기 좋은  ', '설날 선물을 위한 추천  ', '   , 2021년 ‘소중한  들',
       '이제는 버블을 즐길 시간!', '홈스토랑에서 오픈할  은?',
       '   , 프리미티보 & 진판델(Primitivo & Zinfandel)', '기분전환을 위한 추천  ',
       '찬바람이 불 때는 이  을!', '할로윈을 더 재미있게 만들어줄  ', '차박은 이  과 함께!',
       

In [16]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",nan,nan,NaN,NaN,NaN,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,NaN,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",nan,nan,NaN,NaN,NaN,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",nan,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,NaN,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",NaN,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,nan,nan,NaN,NaN,NaN,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",NaN,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,nan,nan,NaN,NaN,NaN,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [17]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name_ko        805 non-null    object
 1   type           68 non-null     object
 2   aroma          321 non-null    object
 3   maker's_note   317 non-null    object
 4   tasting_note   806 non-null    object
 5   posting_title  806 non-null    object
 6   wine_title     290 non-null    object
 7   wine_text      582 non-null    object
 8   hashtag        25 non-null     object
 9   blog_text      301 non-null    object
dtypes: object(10)
memory usage: 63.1+ KB


###### blog_text와 blog_text들을 제외한 모든 열을 합친 새로운 열로 만드는 것이 목표!
###### 이 때, null값이 있으면 합치기 어렵기 때문에 null값에 tokenize 할 때 쉽게 제거할 수 있는 의미 없는 값을 넣음

In [18]:
wine = wine.fillna("ok")

In [19]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",nan,nan,ok,ok,ok,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...
1,루베롱 블랑,ok,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",nan,nan,ok,ok,ok,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",nan,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,ok,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
3,"마스카 델 타코, 로시 피노 네로 로사토",ok,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,nan,nan,ok,ok,ok,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
4,"비에티, 바르베라 다스티 '트레 비그네'",ok,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,nan,nan,ok,ok,ok,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne..."


In [20]:
wine["tasting_note"].isnull().sum()

0

###### 이유는 모르겠으나, fillna로 채워지지 않는 np.nan값은 replace 함수를 써서 대체

In [21]:
wine.replace("nan", "ok", inplace=True)

###### 아래 열들을 합쳐 하나의 열로 만듦

In [22]:
wine["review"] = wine["type"] +"." + wine["aroma"] +"."+ wine["maker's_note"] +"."+ wine["tasting_note"] +"."+ wine["posting_title"] +"."+ wine["wine_title"] +"."+ wine["wine_text"] +"."+ wine["hashtag"] 

In [23]:
wine.head()

,name_ko,type,aroma,maker's_note,tasting_note,posting_title,wine_title,wine_text,hashtag,blog_text,review
0,더 롱 리틀 독 스파클링,스파클링,"복숭아, 살구, 꽃, 흰꽃, 레몬","밝고 맑은 레몬 컬러에 레몬, 살구, 복숭아, 흰꽃의 아로마가 느껴진다. 입 안에서...",ok,ok,ok,ok,ok,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복..."
1,루베롱 블랑,ok,"헤이즐넛, 꽃, 아카시아, 레몬","화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐넛 향이 어우러진다. 입 안에서는 ...",ok,ok,ok,ok,ok,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...,"ok.헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",스파클링,꽃,"반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달콤한 흰 과일...",ok,7월,여왕의 샴페인,1867년 랭스(Reims)에 설립된 마리 스튜아트는 스코틀랜드의 여왕이자 프랑스 ...,ok,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달..."
3,"마스카 델 타코, 로시 피노 네로 로사토",ok,"딸기, 미네랄",밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하고 그 뒤로 향신...,ok,ok,ok,ok,ok,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...,"ok.딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍..."
4,"비에티, 바르베라 다스티 '트레 비그네'",ok,"체리, 오크",바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변인 아그리아노 ...,ok,ok,ok,ok,ok,"(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne...","ok.체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕..."


###### 필요없어진 열 삭제

In [24]:
del wine["type"]
del wine["aroma"]
del wine["maker's_note"]
del wine["tasting_note"]
del wine["wine_title"]
del wine["wine_text"]
del wine["hashtag"]
del wine["posting_title"]

In [25]:
wine.head()

,name_ko,blog_text,review
0,더 롱 리틀 독 스파클링,WINE\n더 롱 리틀 독 스파클링 [ The Long Little Dog Spar...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복..."
1,루베롱 블랑,와인이야기\n[하남 포도내음 와인아울렛/와인샵]와인소개_화이트와인추천_Marreno...,"ok.헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...,"ok.딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍..."
4,"비에티, 바르베라 다스티 '트레 비그네'","(N) Piemonte\nVietti, Barbera d'Asti Tre Vigne...","ok.체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕..."


###### 영어와 기호 의미없는 줄바꿈이 너무 많아 토크나이즈가 제대로 되지 않아, 한글과 숫자 약간의 기호를 제외한 글자들은 모두 삭제함

In [26]:
wine["blog_text"] = wine["blog_text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9,.\s]","")

In [27]:
wine["review"] = wine["review"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9.,\s]","")

In [28]:
wine.head()

,name_ko,blog_text,review
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,".헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,".딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...",".체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변..."


###### review 열을  토큰화한 뒤 원하는 품사를 가진 단어만 남겨둘 새로운 열 추가

In [29]:
wine["review_token"] = " "

In [30]:
wine = wine[0:325]

###### 원하는 품사(의미를 가진 품사)만 새로운 열에 추가함

In [33]:
for i in range(0,325):
    pos = okt.pos(wine.review[i])
    for j in pos:
        if j[1] == "Adjective":
            wine.review_token[i] = wine.review_token[i] + " " + j[0] 
        if j[1] == "Noun":
            wine.review_token[i] = wine.review_token[i] + " " + j[0] 
        if j[1] == "Number":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]     
        if j[1] == "Suffix":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]
        if j[1] == "Verb":
            wine.review_token[i] = wine.review_token[i] + " " + j[0]

In [34]:
wine.head()

,name_ko,blog_text,review,review_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복...",스파클링 복숭아 살구 꽃 흰 꽃 레몬 밝고 맑은 레몬 컬러 레몬 살구 복숭아 흰...
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,".헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐...",헤이즐넛 꽃 아카시아 레몬 화사한 노란색 띠 레몬 아카시아 꽃 헤이즐넛 향 어우...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달...",스파클링 꽃 반짝이는 연한 금빛 정교한 거품 지녔으며 흰 꽃 패션 프루츠 달콤한...
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,".딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하...",딸기 미네랄 밟은 옅은 분홍색 야생 딸기 젤리 수박 같은 달콤한 향 풍부하고 그...
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...",".체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변...",체리 오크 바르 베 다스 티 포도 크 레나 포도 밭 유명한 아스티 언덕 주변인 ...


###### review text 전체의 빈도수를 Dictionary로 나타냄

In [35]:
vocab ={}
for i in range(0,325):
    for word in word_tokenize(wine.review_token[i]):
        if word not in vocab:
            vocab[word] = 0
        vocab[word] +=1

In [36]:
print(vocab)

{'스파클링': 128, '복숭아': 111, '살구': 61, '꽃': 190, '흰': 38, '레몬': 93, '밝고': 12, '맑은': 9, '컬러': 43, '아로마': 181, '느껴진다': 103, '입': 105, '안': 96, '싱그러운': 10, '상큼': 30, '과일': 227, '기분': 47, '좋은': 128, '산미': 70, '포로': 5, '피어나는': 2, '와인': 1054, '헤이즐넛': 18, '아카시아': 23, '화사한': 7, '노란색': 16, '띠': 147, '향': 574, '어우러진다': 10, '부드럽고': 33, '섬세한': 51, '맛': 132, '생기': 10, '있는': 151, '피니시': 18, '인상': 43, '적': 354, '반짝이는': 15, '연한': 20, '금빛': 10, '정교한': 4, '거품': 10, '지녔으며': 3, '패션': 6, '프루츠': 3, '달콤한': 52, '은은한': 32, '미네럴': 2, '느낄': 84, '수': 193, '있다': 221, '우아하고': 18, '균형': 60, '감': 90, '뛰어난': 63, '샴페인': 97, '발레': 3, '드': 31, '마른': 4, '과': 13, '세잔': 2, '생산': 224, '최상급': 10, '피노': 84, '뮈니': 7, '누': 67, '샤르도네': 26, '품종': 106, '만든': 38, '마리': 6, '스튜': 8, '아트': 6, '리지': 5, '널': 6, '블렌딩': 30, '비율': 8, '한다': 118, '뀌베': 9, '헨느': 3, '여왕': 5, '별칭': 3, '갖고': 4, '있으며': 66, '도사': 4, '주는': 27, '8': 16, '7월': 6, '1867년': 2, '랭스': 1, '설립': 35, '된': 147, '스코틀랜드': 2, '이자': 8, '프랑스': 50, '왕비': 1, '이름': 58, '딴': 3, '하우스': 21

In [37]:
print(vocab)

{'스파클링': 128, '복숭아': 111, '살구': 61, '꽃': 190, '흰': 38, '레몬': 93, '밝고': 12, '맑은': 9, '컬러': 43, '아로마': 181, '느껴진다': 103, '입': 105, '안': 96, '싱그러운': 10, '상큼': 30, '과일': 227, '기분': 47, '좋은': 128, '산미': 70, '포로': 5, '피어나는': 2, '와인': 1054, '헤이즐넛': 18, '아카시아': 23, '화사한': 7, '노란색': 16, '띠': 147, '향': 574, '어우러진다': 10, '부드럽고': 33, '섬세한': 51, '맛': 132, '생기': 10, '있는': 151, '피니시': 18, '인상': 43, '적': 354, '반짝이는': 15, '연한': 20, '금빛': 10, '정교한': 4, '거품': 10, '지녔으며': 3, '패션': 6, '프루츠': 3, '달콤한': 52, '은은한': 32, '미네럴': 2, '느낄': 84, '수': 193, '있다': 221, '우아하고': 18, '균형': 60, '감': 90, '뛰어난': 63, '샴페인': 97, '발레': 3, '드': 31, '마른': 4, '과': 13, '세잔': 2, '생산': 224, '최상급': 10, '피노': 84, '뮈니': 7, '누': 67, '샤르도네': 26, '품종': 106, '만든': 38, '마리': 6, '스튜': 8, '아트': 6, '리지': 5, '널': 6, '블렌딩': 30, '비율': 8, '한다': 118, '뀌베': 9, '헨느': 3, '여왕': 5, '별칭': 3, '갖고': 4, '있으며': 66, '도사': 4, '주는': 27, '8': 16, '7월': 6, '1867년': 2, '랭스': 1, '설립': 35, '된': 147, '스코틀랜드': 2, '이자': 8, '프랑스': 50, '왕비': 1, '이름': 58, '딴': 3, '하우스': 21

###### 빈도수 순으로 나열함

In [38]:
vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

In [39]:
print(vocab_sorted)

[('와인', 1054), ('향', 574), ('적', 354), ('오크', 256), ('포도', 253), ('과일', 227), ('생산', 224), ('있다', 221), ('체리', 213), ('잘', 196), ('수', 193), ('꽃', 190), ('아로마', 181), ('과실', 155), ('이', 153), ('있는', 151), ('띠', 147), ('된', 147), ('자두', 146), ('숙성', 144), ('등', 144), ('맛', 132), ('스파클링', 128), ('좋은', 128), ('지역', 127), ('하는', 126), ('한다', 118), ('복숭아', 111), ('배', 109), ('바닐라', 107), ('품종', 106), ('입', 105), ('해', 104), ('부드러운', 104), ('느껴진다', 103), ('들', 103), ('샴페인', 97), ('안', 96), ('풍부한', 96), ('너리', 96), ('붉은', 94), ('타닌', 94), ('레몬', 93), ('산도', 92), ('미네랄', 91), ('감', 90), ('색', 90), ('느낌', 90), ('여운', 89), ('익은', 89), ('신선한', 86), ('블랙', 85), ('느낄', 84), ('피노', 84), ('사용', 83), ('사과', 83), ('로', 82), ('발효', 82), ('통', 81), ('의', 79), ('딸기', 78), ('바디', 78), ('하며', 76), ('매력', 75), ('그', 74), ('미가', 73), ('후', 73), ('수확', 73), ('개월', 72), ('트러스', 72), ('블랙베리', 71), ('산미', 70), ('입안', 70), ('것', 70), ('빛', 69), ('같은', 68), ('누', 67), ('루비', 67), ('있으며', 66), ('레드', 66), ('복합', 66)

###### 빈도수가 1이하인 단어 삭제

In [40]:
word_to_index = {}
i = 0
for word, frequency in vocab_sorted:
    if frequency > 1:
        i += 1
        word_to_index[word] = i

In [41]:
print(word_to_index)

{'와인': 1, '향': 2, '적': 3, '오크': 4, '포도': 5, '과일': 6, '생산': 7, '있다': 8, '체리': 9, '잘': 10, '수': 11, '꽃': 12, '아로마': 13, '과실': 14, '이': 15, '있는': 16, '띠': 17, '된': 18, '자두': 19, '숙성': 20, '등': 21, '맛': 22, '스파클링': 23, '좋은': 24, '지역': 25, '하는': 26, '한다': 27, '복숭아': 28, '배': 29, '바닐라': 30, '품종': 31, '입': 32, '해': 33, '부드러운': 34, '느껴진다': 35, '들': 36, '샴페인': 37, '안': 38, '풍부한': 39, '너리': 40, '붉은': 41, '타닌': 42, '레몬': 43, '산도': 44, '미네랄': 45, '감': 46, '색': 47, '느낌': 48, '여운': 49, '익은': 50, '신선한': 51, '블랙': 52, '느낄': 53, '피노': 54, '사용': 55, '사과': 56, '로': 57, '발효': 58, '통': 59, '의': 60, '딸기': 61, '바디': 62, '하며': 63, '매력': 64, '그': 65, '미가': 66, '후': 67, '수확': 68, '개월': 69, '트러스': 70, '블랙베리': 71, '산미': 72, '입안': 73, '것': 74, '빛': 75, '같은': 76, '누': 77, '루비': 78, '있으며': 79, '레드': 80, '복합': 81, '우아한': 82, '뛰어난': 83, '개': 84, '가지': 85, '살구': 86, '도': 87, '균형': 88, '오렌지': 89, '는': 90, '이름': 91, '특징': 92, '가장': 93, '느껴지는': 94, '진한': 95, '긴': 96, '미와': 97, '조화': 98, '품질': 99, '허브': 100, '밭': 101, '화이트

###### 인코딩한 결과가 들어갈 새로운 열 만듦

In [43]:
wine["sorted_review_token"] = " "

###### 중복 제거, 빈도수에 다른 우선순위대로 나열 

In [44]:
for i in range(0,325):
    sentence=[]
    for word in word_tokenize(wine.review_token[i]):
         if word in word_to_index.keys():
                if word_to_index[word] not in sentence:
                    sentence.append(word_to_index[word])
    sentence = sorted(sentence)
    wine["sorted_review_token"][i] = sentence

In [46]:
wine.head()

,name_ko,blog_text,review,review_token,sorted_review_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복...",스파클링 복숭아 살구 꽃 흰 꽃 레몬 밝고 맑은 레몬 컬러 레몬 살구 복숭아 흰...,"[1, 6, 12, 13, 23, 24, 28, 32, 35, 38, 43, 72,..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,".헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐...",헤이즐넛 꽃 아카시아 레몬 화사한 노란색 띠 레몬 아카시아 꽃 헤이즐넛 향 어우...,"[1, 2, 3, 12, 16, 17, 22, 32, 38, 43, 104, 127..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달...",스파클링 꽃 반짝이는 연한 금빛 정교한 거품 지녔으며 흰 꽃 패션 프루츠 달콤한...,"[6, 7, 8, 11, 12, 18, 23, 26, 27, 31, 36, 37, ..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,".딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하...",딸기 미네랄 밟은 옅은 분홍색 야생 딸기 젤리 수박 같은 달콤한 향 풍부하고 그...,"[1, 2, 14, 32, 38, 45, 49, 61, 65, 76, 82, 103..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...",".체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변...",체리 오크 바르 베 다스 티 포도 크 레나 포도 밭 유명한 아스티 언덕 주변인 ...,"[2, 4, 5, 7, 9, 17, 20, 25, 35, 41, 42, 44, 47..."


###### value 값으로 key값을 찾기 위해 dictionary를 뒤집음

In [47]:
word_to_index_reverse = {v:k for k,v in word_to_index.items()}

###### 인코딩이  따로 필요 없기 때문에 한국어로 다시 원상복귀함

In [48]:
for i in range(0,325):
    sentence=[]
    for word in wine.sorted_review_token[i]:
         sentence.append(word_to_index_reverse[word])
    wine["review_token"][i] = sentence

In [50]:
wine.head()

,name_ko,blog_text,review,review_token,sorted_review_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"스파클링.복숭아, 살구, 꽃, 흰꽃, 레몬.밝고 맑은 레몬 컬러에 레몬, 살구, 복...","[와인, 과일, 꽃, 아로마, 스파클링, 좋은, 복숭아, 입, 느껴진다, 안, 레몬...","[1, 6, 12, 13, 23, 24, 28, 32, 35, 38, 43, 72,..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,".헤이즐넛, 꽃, 아카시아, 레몬.화사한 노란색을 띠며 레몬, 아카시아 꽃, 헤이즐...","[와인, 향, 적, 꽃, 있는, 띠, 맛, 입, 안, 레몬, 섬세한, 인상, 부드럽...","[1, 2, 3, 12, 16, 17, 22, 32, 38, 43, 104, 127..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"스파클링.꽃.반짝이는 연한 금빛의 정교한 거품을 지녔으며, 흰 꽃 패션 프루츠, 달...","[과일, 생산, 있다, 수, 꽃, 된, 스파클링, 하는, 한다, 품종, 들, 샴페인...","[6, 7, 8, 11, 12, 18, 23, 26, 27, 31, 36, 37, ..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,".딸기, 미네랄.밟은 옅은 분홍색에 야생 딸기 젤리 수박과 같은 달콤한 향이 풍부하...","[와인, 향, 과실, 입, 안, 미네랄, 여운, 딸기, 그, 같은, 우아한, 달콤한...","[1, 2, 14, 32, 38, 45, 49, 61, 65, 76, 82, 103..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...",".체리, 오크.바르베라 다스티의 포도는 라 크레나 포도밭으로 유명한 아스티 언덕주변...","[향, 오크, 포도, 생산, 체리, 띠, 숙성, 지역, 느껴진다, 붉은, 타닌, 산...","[2, 4, 5, 7, 9, 17, 20, 25, 35, 41, 42, 44, 47..."


###### 필요 없어진 열 삭제

In [51]:
del wine["review"]
del wine["sorted_review_token"]

In [52]:
wine.head()

,name_ko,blog_text,review_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"[와인, 과일, 꽃, 아로마, 스파클링, 좋은, 복숭아, 입, 느껴진다, 안, 레몬..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,"[와인, 향, 적, 꽃, 있는, 띠, 맛, 입, 안, 레몬, 섬세한, 인상, 부드럽..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"[과일, 생산, 있다, 수, 꽃, 된, 스파클링, 하는, 한다, 품종, 들, 샴페인..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,"[와인, 향, 과실, 입, 안, 미네랄, 여운, 딸기, 그, 같은, 우아한, 달콤한..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...","[향, 오크, 포도, 생산, 체리, 띠, 숙성, 지역, 느껴진다, 붉은, 타닌, 산..."


###### 위와 마찬가지로 blog_text 열을  토큰화한 뒤 원하는 품사를 가진 단어만 남겨둘 새로운 열 추가

In [53]:
wine["blog_token"] = " "

###### 원하는 품사(의미를 가진 품사)만 새로운 열에 추가함

In [54]:
pos_okt = okt.pos(wine.blog_text[1], norm=True, stem=True)

In [55]:
for i in range(0,325):
    pos = okt.pos(wine.blog_text[i], norm=True, stem=True)
    for j in pos:
        if j[1] == "Verb":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 
        if j[1] == "Noun":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 
        if j[1] == "Adjective":
            wine.blog_token[i] = wine.blog_token[i] + " " + j[0] 

In [56]:
wine.head()

,name_ko,blog_text,review_token,blog_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"[와인, 과일, 꽃, 아로마, 스파클링, 좋은, 복숭아, 입, 느껴진다, 안, 레몬...",더 롱 리틀 독 스파클링 소유리 복사 이웃 추가 더 롱 리틀 독 스파클링 단맛 ...
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,"[와인, 향, 적, 꽃, 있는, 띠, 맛, 입, 안, 레몬, 섬세한, 인상, 부드럽...",와인 이야기 하남 포도 음 와인 아울렛 와인 와인 소개 화이트와인 추천 마레 농...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"[과일, 생산, 있다, 수, 꽃, 된, 스파클링, 하는, 한다, 품종, 들, 샴페인...",와인 추천 와인 컴 선정 달 와인 와인 복사 이웃 추가 조금 늦다 찾아오다 장마...
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,"[와인, 향, 과실, 입, 안, 미네랄, 여운, 딸기, 그, 같은, 우아한, 달콤한...",이탈리아 와인 마 스카 델 타코 제이 픽 와인 산타 복사 이웃 추가 마 스카 델...
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...","[향, 오크, 포도, 생산, 체리, 띠, 숙성, 지역, 느껴진다, 붉은, 타닌, 산...",비 띠 바르다 베 다스 티 트레 비녜 데미안 복사 이웃 추가 비 띠 바르다 베 ...


##### 혹시 모를일에 대비해서 복사본을 만들어뒀습니다

In [57]:
wine_copy = wine.copy()

###### review_text와 마찬가지로 많이 단어 dictionary를 만들었습니다.

In [58]:
blog_vocab ={}
for i in range(0,325):
    for word in word_tokenize(wine_copy.blog_token[i]):
        if word not in blog_vocab:
            blog_vocab[word] = 0
        blog_vocab[word] +=1

In [59]:
print(blog_vocab)

{'더': 2942, '롱': 212, '리틀': 49, '독': 294, '스파클링': 1919, '소유리': 1, '복사': 1703, '이웃': 1784, '추가': 1968, '단맛': 355, '없다': 2130, '깔끔하다': 410, '와인': 42943, '검색': 133, '해보다': 587, '로': 2921, '제나': 8, '루즈': 362, '마시다': 2017, '것': 4560, '같다': 3023, '대한': 768, '정보': 421, '아예': 33, '어플': 12, '이': 6266, '날': 1277, '엄마': 69, '아빠': 26, '결혼기념일': 14, '가족': 578, '가볍다': 849, '드라이': 816, '과일': 1694, '마시기': 568, '좋다': 5953, '전혀': 172, '페어': 447, '링': 716, '되다': 8841, '않다': 3740, '음식': 1262, '이러하다': 168, '하다': 32212, '출처': 336, '나': 933, '평점': 276, '가격': 2373, '입처': 385, '레드': 2694, '텅부띠': 1, '크다': 1863, '블': 2045, '훈장': 3, '프랑스': 4061, '빈티': 3630, '구입': 664, '왜': 173, '하프': 49, '보틀': 410, '사실': 417, '늘다': 1919, '맞다': 530, '요즘': 291, '보다': 3846, '병': 1505, '정도': 1939, '그렇다': 1076, '남다': 432, '어': 66, '캐다': 23, '주로': 363, '다음': 487, '그': 3873, '말다': 1930, '산화': 102, '맛': 3115, '변화': 184, '느끼다': 1152, '본의': 10, '아니다': 1722, '경우': 472, '있다': 16678, '때': 1718, '필요하다': 246, '바로': 932, '거': 593, '빈병': 6, '채우다':

##### 많이 사용된 단어 순으로 나열했습니다.

In [60]:
blog_vocab_sorted = sorted(blog_vocab.items(), key=lambda x: x[1], reverse=True)

In [61]:
print(blog_vocab_sorted)

[('와인', 42943), ('하다', 32212), ('있다', 16678), ('되다', 8841), ('향', 7278), ('이', 6266), ('좋다', 5953), ('품종', 5850), ('너리', 5745), ('이다', 5425), ('포도', 5400), ('생산', 4908), ('수', 4704), ('것', 4560), ('점', 4437), ('지역', 4174), ('프랑스', 4061), ('그', 3873), ('보다', 3846), ('않다', 3740), ('빈티', 3630), ('자다', 3626), ('점수', 3475), ('비비다', 3438), ('이탈리아', 3344), ('피다', 3292), ('등', 3164), ('맛', 3115), ('같다', 3023), ('만들다', 2979), ('더', 2942), ('로', 2921), ('누', 2817), ('소비', 2803), ('샴페인', 2768), ('미국', 2761), ('되어다', 2757), ('의', 2734), ('레드', 2694), ('오다', 2668), ('뇽', 2653), ('바', 2569), ('도', 2561), ('입사', 2505), ('밉다', 2463), ('가다', 2452), ('화이트', 2379), ('가격', 2373), ('느낌', 2364), ('드', 2271), ('느껴지다', 2269), ('숙성', 2267), ('국가', 2239), ('중', 2238), ('오크', 2160), ('없다', 2130), ('비', 2108), ('받다', 2076), ('리', 2049), ('블', 2045), ('마시다', 2017), ('가장', 2000), ('추가', 1968), ('과실', 1955), ('정도', 1939), ('스페인', 1936), ('말다', 1930), ('스파클링', 1919), ('늘다', 1919), ('끄다', 1879), ('추천', 1875), ('까베르', 

###### 단어의 빈도수를 확인하고, 블로그의 특성상 존재하는 관련없는 말, 의미없는 말을 제거하기 위해 빈도수가 3이 넘는 값만 중복을 제거하여 남겨둠

In [62]:
blog_word_to_index = {}
i = 0
for word, frequency in blog_vocab_sorted:
    if frequency > 3:
        i += 1
        blog_word_to_index[word] = i

In [63]:
print(blog_word_to_index)

{'와인': 1, '하다': 2, '있다': 3, '되다': 4, '향': 5, '이': 6, '좋다': 7, '품종': 8, '너리': 9, '이다': 10, '포도': 11, '생산': 12, '수': 13, '것': 14, '점': 15, '지역': 16, '프랑스': 17, '그': 18, '보다': 19, '않다': 20, '빈티': 21, '자다': 22, '점수': 23, '비비다': 24, '이탈리아': 25, '피다': 26, '등': 27, '맛': 28, '같다': 29, '만들다': 30, '더': 31, '로': 32, '누': 33, '소비': 34, '샴페인': 35, '미국': 36, '되어다': 37, '의': 38, '레드': 39, '오다': 40, '뇽': 41, '바': 42, '도': 43, '입사': 44, '밉다': 45, '가다': 46, '화이트': 47, '가격': 48, '느낌': 49, '드': 50, '느껴지다': 51, '숙성': 52, '국가': 53, '중': 54, '오크': 55, '없다': 56, '비': 57, '받다': 58, '리': 59, '블': 60, '마시다': 61, '가장': 62, '추가': 63, '과실': 64, '정도': 65, '스페인': 66, '말다': 67, '스파클링': 68, '늘다': 69, '끄다': 70, '추천': 71, '까베르': 72, '크다': 73, '사용': 74, '이웃': 75, '메다': 76, '밭': 77, '아니다': 78, '때': 79, '판매': 80, '복사': 81, '과일': 82, '들다': 83, '데': 84, '산도': 85, '레': 86, '먹다': 87, '부드럽다': 88, '즐기다': 89, '밸리': 90, '병': 91, '곳': 92, '위': 93, '높다': 94, '를': 95, '많다': 96, '바디': 97, '이름': 98, '로제': 99, '감': 100, '어울리다': 101, '후':

In [64]:
wine_copy["sorted_blog_token"] = " "

###### blog_token을 빈도수대로 인코딩했습니다

In [65]:
for i in range(0,325):
    sentence=[]
    for word in word_tokenize(wine_copy.blog_token[i]):
         if word in blog_word_to_index.keys():
                if blog_word_to_index[word] not in sentence:
                    sentence.append(blog_word_to_index[word])
    sentence = sorted(sentence)
    wine_copy["sorted_blog_token"][i] = sentence

In [66]:
wine_copy.head()

,name_ko,blog_text,review_token,blog_token,sorted_blog_token
0,더 롱 리틀 독 스파클링,\n더 롱 리틀 독 스파클링 \n소유리 2021. 6. 29. 1549...,"[와인, 과일, 꽃, 아로마, 스파클링, 좋은, 복숭아, 입, 느껴진다, 안, 레몬...",더 롱 리틀 독 스파클링 소유리 복사 이웃 추가 더 롱 리틀 독 스파클링 단맛 ...,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1..."
1,루베롱 블랑,와인이야기\n하남 포도내음 와인아울렛와인샵와인소개화이트와인추천 마레농 클래식 루...,"[와인, 향, 적, 꽃, 있는, 띠, 맛, 입, 안, 레몬, 섬세한, 인상, 부드럽...",와인 이야기 하남 포도 음 와인 아울렛 와인 와인 소개 화이트와인 추천 마레 농...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 2021. 7. 1....,"[과일, 생산, 있다, 수, 꽃, 된, 스파클링, 하는, 한다, 품종, 들, 샴페인...",와인 추천 와인 컴 선정 달 와인 와인 복사 이웃 추가 조금 늦다 찾아오다 장마...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,"마스카 델 타코, 로시 피노 네로 로사토",이탈리아와인\n 마스카 델 타코 제이픽와인\n산타 2021. 6. 21. 20...,"[와인, 향, 과실, 입, 안, 미네랄, 여운, 딸기, 그, 같은, 우아한, 달콤한...",이탈리아 와인 마 스카 델 타코 제이 픽 와인 산타 복사 이웃 추가 마 스카 델...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,"비에티, 바르베라 다스티 '트레 비그네'","\n, 2018 비에띠, 바르베라 다스티 트레 비녜 2018\n데미안 2...","[향, 오크, 포도, 생산, 체리, 띠, 숙성, 지역, 느껴진다, 붉은, 타닌, 산...",비 띠 바르다 베 다스 티 트레 비녜 데미안 복사 이웃 추가 비 띠 바르다 베 ...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [67]:
blog_word_to_index_reverse = {v:k for k,v in blog_word_to_index.items()}
print(blog_word_to_index_reverse)

{1: '와인', 2: '하다', 3: '있다', 4: '되다', 5: '향', 6: '이', 7: '좋다', 8: '품종', 9: '너리', 10: '이다', 11: '포도', 12: '생산', 13: '수', 14: '것', 15: '점', 16: '지역', 17: '프랑스', 18: '그', 19: '보다', 20: '않다', 21: '빈티', 22: '자다', 23: '점수', 24: '비비다', 25: '이탈리아', 26: '피다', 27: '등', 28: '맛', 29: '같다', 30: '만들다', 31: '더', 32: '로', 33: '누', 34: '소비', 35: '샴페인', 36: '미국', 37: '되어다', 38: '의', 39: '레드', 40: '오다', 41: '뇽', 42: '바', 43: '도', 44: '입사', 45: '밉다', 46: '가다', 47: '화이트', 48: '가격', 49: '느낌', 50: '드', 51: '느껴지다', 52: '숙성', 53: '국가', 54: '중', 55: '오크', 56: '없다', 57: '비', 58: '받다', 59: '리', 60: '블', 61: '마시다', 62: '가장', 63: '추가', 64: '과실', 65: '정도', 66: '스페인', 67: '말다', 68: '스파클링', 69: '늘다', 70: '끄다', 71: '추천', 72: '까베르', 73: '크다', 74: '사용', 75: '이웃', 76: '메다', 77: '밭', 78: '아니다', 79: '때', 80: '판매', 81: '복사', 82: '과일', 83: '들다', 84: '데', 85: '산도', 86: '레', 87: '먹다', 88: '부드럽다', 89: '즐기다', 90: '밸리', 91: '병', 92: '곳', 93: '위', 94: '높다', 95: '를', 96: '많다', 97: '바디', 98: '이름', 99: '로제', 100: '감', 101: '어울리다', 102:

In [68]:
for i in range(0,325):
    sentence=[]
    for word in wine_copy["sorted_blog_token"][i]:
         sentence.append(blog_word_to_index_reverse[word])
    wine_copy["blog_token"][i] = sentence

In [70]:
wine = wine_copy.copy()

In [72]:
del wine["blog_text"]
del wine["sorted_blog_token"]

In [78]:
wine.head()

,name_ko,review_token,blog_token
0,더 롱 리틀 독 스파클링,"[와인, 과일, 꽃, 아로마, 스파클링, 좋은, 복숭아, 입, 느껴진다, 안, 레몬...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 이다, 포도, 생산, 수, ..."
1,루베롱 블랑,"[와인, 향, 적, 꽃, 있는, 띠, 맛, 입, 안, 레몬, 섬세한, 인상, 부드럽...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 너리, 이다, 포도, 생산,..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","[과일, 생산, 있다, 수, 꽃, 된, 스파클링, 하는, 한다, 품종, 들, 샴페인...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 너리, 이다, 포도, 생산,..."
3,"마스카 델 타코, 로시 피노 네로 로사토","[와인, 향, 과실, 입, 안, 미네랄, 여운, 딸기, 그, 같은, 우아한, 달콤한...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 너리, 이다, 포도, 생산,..."
4,"비에티, 바르베라 다스티 '트레 비그네'","[향, 오크, 포도, 생산, 체리, 띠, 숙성, 지역, 느껴진다, 붉은, 타닌, 산...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 너리, 이다, 포도, 생산,..."


In [84]:
from nltk.corpus import stopwords

###### 많이 사용되었기는 하나, 너무 많이 사용되었기에 의미가 없는 말(ex.와인_공통특성)이나 품사에서 미처 걸러지지 않은 의미없는 단어를 stopword 리스트로 만들었습니다.

In [86]:
review_stop_word = ["와인", "적", "있다", "잘", "수", "이", "있는", "띠", "된", "등", "하는", "한다", "해", "들", "안", "너리", "피노", "로", "의", "하며","미가", "후", "같은", "밭", "할", "되는", "된다", "중", "월간", "이상", "를", "약간", "동안", "매우", "했다", "되었다", "통해", "뒤", "및", "있고", "입니다", "듯", "오슈", "되고", "자", "만", "페", "멘", "까"]

In [87]:
blog_stop_word =["와인", "하다", "있다", "되다", "이", "품종", "너리", "이다", "수", "것", "점", "지역", "그", "보다", "않다", "자다", "점수", "비비다", "피다", "등", "맛", "같다", "만들다", "더", "로", "누", "소비", "샴페인", "되어다", "의", "오다", "뇽", "바", "도", "입사", "밉다", "가다", "가격", "드", "느낌", "느껴지다", "국가", "중" , "없다", "비", "받다", "리", "블", "마시다", "가장", "추가", "정도", "말다", "늘다", "끄다","사용", "이웃", "메다", "밭", "아니다", "때", "판매", "복사", "들다", "데", "레", "먹다", "밸리", "병", "곳", "위", "높다", "를", "많다",
"이름", "감", "후", "재배", "샤",  "생각", "처", "음", "시작", "날" , "개", "입", "트", "시라", "다른", "시간", "뒤", "전", "테이", "그렇다", "및","스", "티", "지다", "메", "싶다", "이상", "약간", "내다", "듯", "코", "쉬라즈", "하나", "특징", "주다", "때문", "스팅", "디", "좀", "아주", "나", "바로", "위해", "알다", "테이블", "안", "빈야드", "제", "내", "풀", "나다", "자리", "슬링", "함","론", "알", "브", "또", "은", "소개", "보이다", "링", "색", "이렇다",  "베", "두", "번", "제르", "이후", "포", "조금", "테", "시", "해주다", "일", "르", "통", "종", "정말", "저"]

###### blog와 review 열을 합칠 새로운 열을 만들어줍니다.

In [79]:
wine["blogreview"] = " "

###### list를 스트링으로 바꿔야 tokenizing이 진행될 수 있기 때문에 blog_token과 review_token 모두 새로운 열을 만들고 str으로 바꿔줍니다.

In [93]:
wine["review"] = " "

In [99]:
for i in range(0,325):
    wine["review"][i] = " ".join(wine['review_token'][i])

In [101]:
wine["blog"] = " "

In [102]:
for i in range(0,325):
    wine["blog"][i] = " ".join(wine['blog_token'][i])

###### 합치면서 stopword에 있는 말은 빼고, review와 blog에서 중복되는 값을 모두 제외했습니다.

In [109]:
for i in range(0,325):  
    sentence=[]
    for word in word_tokenize(wine.review[i]):
         if word not in review_stop_word:
                if word not in sentence:
                    sentence.append(word)
    for word in word_tokenize(wine.blog[i]):
         if word not in blog_stop_word:
                if word not in sentence:
                    sentence.append(word)

    wine["blogreview"][i] = sentence

In [110]:
wine.head()

,name_ko,review_token,blog_token,blogreview,review,blog
0,더 롱 리틀 독 스파클링,"[와인, 과일, 꽃, 아로마, 스파클링, 좋은, 복숭아, 입, 느껴진다, 안, 레몬...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 이다, 포도, 생산, 수, ...","[과일, 꽃, 아로마, 스파클링, 좋은, 복숭아, 입, 느껴진다, 레몬, 산미, 살...",와인 과일 꽃 아로마 스파클링 좋은 복숭아 입 느껴진다 안 레몬 산미 살구 기분 컬...,와인 하다 있다 되다 향 이 좋다 품종 이다 포도 생산 수 것 지역 프랑스 그 보다...
1,루베롱 블랑,"[와인, 향, 적, 꽃, 있는, 띠, 맛, 입, 안, 레몬, 섬세한, 인상, 부드럽...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 너리, 이다, 포도, 생산,...","[향, 꽃, 맛, 입, 레몬, 섬세한, 인상, 부드럽고, 아카시아, 헤이즐넛, 피니...",와인 향 적 꽃 있는 띠 맛 입 안 레몬 섬세한 인상 부드럽고 아카시아 헤이즐넛 피...,와인 하다 있다 되다 향 이 좋다 품종 너리 이다 포도 생산 수 것 점 지역 프랑스...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","[과일, 생산, 있다, 수, 꽃, 된, 스파클링, 하는, 한다, 품종, 들, 샴페인...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 너리, 이다, 포도, 생산,...","[과일, 생산, 꽃, 스파클링, 품종, 샴페인, 감, 느낄, 그, 누, 있으며, 우...",과일 생산 있다 수 꽃 된 스파클링 하는 한다 품종 들 샴페인 감 느낄 피노 로 하...,와인 하다 있다 되다 향 이 좋다 품종 너리 이다 포도 생산 수 것 점 지역 프랑스...
3,"마스카 델 타코, 로시 피노 네로 로사토","[와인, 향, 과실, 입, 안, 미네랄, 여운, 딸기, 그, 같은, 우아한, 달콤한...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 너리, 이다, 포도, 생산,...","[향, 과실, 입, 미네랄, 여운, 딸기, 그, 우아한, 달콤한, 향신료, 부드럽고...",와인 향 과실 입 안 미네랄 여운 딸기 그 같은 우아한 달콤한 뒤 향신료 부드럽고 ...,와인 하다 있다 되다 향 이 좋다 품종 너리 이다 포도 생산 수 것 점 지역 프랑스...
4,"비에티, 바르베라 다스티 '트레 비그네'","[향, 오크, 포도, 생산, 체리, 띠, 숙성, 지역, 느껴진다, 붉은, 타닌, 산...","[와인, 하다, 있다, 되다, 향, 이, 좋다, 품종, 너리, 이다, 포도, 생산,...","[향, 오크, 포도, 생산, 체리, 숙성, 지역, 느껴진다, 붉은, 타닌, 산도, ...",향 오크 포도 생산 체리 띠 숙성 지역 느껴진다 붉은 타닌 산도 색 여운 익은 통 ...,와인 하다 있다 되다 향 이 좋다 품종 너리 이다 포도 생산 수 것 점 지역 프랑스...


###### 필요없어진 열을 모두 제거했습니다

In [112]:
del wine["review_token"]
del wine["blog_token"]
del wine["review"]
del wine["blog"]

In [113]:
wine.head()

,name_ko,blogreview
0,더 롱 리틀 독 스파클링,"[과일, 꽃, 아로마, 스파클링, 좋은, 복숭아, 입, 느껴진다, 레몬, 산미, 살..."
1,루베롱 블랑,"[향, 꽃, 맛, 입, 레몬, 섬세한, 인상, 부드럽고, 아카시아, 헤이즐넛, 피니..."
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼","[과일, 생산, 꽃, 스파클링, 품종, 샴페인, 감, 느낄, 그, 누, 있으며, 우..."
3,"마스카 델 타코, 로시 피노 네로 로사토","[향, 과실, 입, 미네랄, 여운, 딸기, 그, 우아한, 달콤한, 향신료, 부드럽고..."
4,"비에티, 바르베라 다스티 '트레 비그네'","[향, 오크, 포도, 생산, 체리, 숙성, 지역, 느껴진다, 붉은, 타닌, 산도, ..."


###### blogreview열도 가공하기 쉽게 list에서 str으로 변경했습니다.

In [114]:
for i in range(0,325):
  wine["blogreview"][i] = " ".join(wine['blogreview'][i])

In [115]:
wine.head()

,name_ko,blogreview
0,더 롱 리틀 독 스파클링,과일 꽃 아로마 스파클링 좋은 복숭아 입 느껴진다 레몬 산미 살구 기분 컬러 흰 상...
1,루베롱 블랑,향 꽃 맛 입 레몬 섬세한 인상 부드럽고 아카시아 헤이즐넛 피니시 노란색 어우러진다...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",과일 생산 꽃 스파클링 품종 샴페인 감 느낄 그 누 있으며 우아한 뛰어난 균형 이름...
3,"마스카 델 타코, 로시 피노 네로 로사토",향 과실 입 미네랄 여운 딸기 그 우아한 달콤한 향신료 부드럽고 풍부하고 옅은 야생...
4,"비에티, 바르베라 다스티 '트레 비그네'",향 오크 포도 생산 체리 숙성 지역 느껴진다 붉은 타닌 산도 색 여운 익은 통 입안...


###### csv파일로 저장해 modeling팀으로 옮겼습니다.

In [116]:
wine.to_csv("wine_tokenizing_final")